In [75]:
import requests
import csv

In [76]:
api_ship = "https://mixivivu.com/api/products/get-list?size=5&page="
api_features = "https://mixivivu.com/api/features/get-list"
api_room_features = "https://mixivivu.com/api/features/get-list?size=30"
page = 1

In [77]:
response = requests.get(api_features)
response.raise_for_status()
features_data = response.json().get("result", {}).get("data", [])
feature_dict = {feature["_id"]: feature["text"] for feature in features_data}

In [78]:
response = requests.get(api_room_features)
response.raise_for_status()
room_features_data = response.json().get("result", {}).get("data", [])
room_feature_dict = {feature["_id"]: feature["text"] for feature in room_features_data}

In [79]:
with open("../data/ship.csv", "w", encoding="utf-8", newline="") as ship_file, \
     open("../data/ship_rooms.csv", "w", encoding="utf-8", newline="") as room_file:

    ship_writer = csv.writer(ship_file)
    room_writer = csv.writer(room_file)

    ship_writer.writerow(["ship_id", "ship_name", "launch", "cabin", "shell", "trip", "admin", "ship_price", "address", "map_link", "ship_features", "short_description", "long_description"])
    room_writer.writerow(["room_id", "ship_id", "room_name", "size", "max_persons", "room_price", "room_features", ])

    ship_id = 1
    room_id = 1
    while True:
        try:
            response = requests.get(api_ship + str(page))
            response.raise_for_status()

            data = response.json()
            ships = data.get("result", {}).get("data", [])

            if not ships:
                print(f"No more data on page {page}. Stopping...")
                break

            for ship in ships:
                ship_name = ship.get("title")
                launch = ship.get("spec", {}).get("ship", {}).get("launch")
                cabin = ship.get("spec", {}).get("ship", {}).get("cabin")
                shell = ship.get("spec", {}).get("ship", {}).get("shell")
                trip = ship.get("spec", {}).get("ship", {}).get("trip")
                admin = ship.get("spec", {}).get("ship", {}).get("admin")
                ship_price = ship.get("defaultPrice")
                address = ship.get("address")
                map_link = ship.get("mapLink")

                ship_features_list = ship.get("features", [])
                ship_features = ", ".join(feature_dict.get(feature, feature) for feature in ship_features_list if feature in feature_dict)

                short_description = ship.get("shortDescription")
                long_description = ship.get("longDescription")
                rooms = ship.get("rooms")

                ship_writer.writerow([ship_id, ship_name, launch, cabin, shell, trip, admin, ship_price, address, map_link, ship_features, short_description, long_description])

                for room in rooms:
                    room_name = room.get("title")
                    room_price = room.get("price")
                    size = room.get("size")
                    max_persons = room.get("maxPersons")

                    room_features_list = room.get("features", [])
                    room_features = ", ".join(room_feature_dict.get(feature, feature) for feature in room_features_list)

                    room_writer.writerow([room_id, ship_id, room_name, size, max_persons, room_price, room_features])
                    room_id += 1

                ship_id += 1

            page += 1
        except requests.RequestException as e:
            print(f"Error while fetching page {page}: {e}")
            break

print("Data crawling and saving completed!")

No more data on page 15. Stopping...
Data crawling and saving completed!
